In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import os
import warnings
from machine_learning.utils import scale_dataset, get_distribution, plot_distribution
from machine_learning.neural_networks.utils import plot_history, split_data
from machine_learning.neural_networks.deep_fnn import train_deep_fnn_model
from configs.enums import Column, RiskClassifications
import numpy as np
import matplotlib.pyplot as plt
from configs.data import MACHINE_LEARNING_DATASET_PATH, MERGED_DATASET_PATH, OUT_PATH, MODELS_PATH, VERSION
import shap
import tensorflow as tf
from typing import Tuple

warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(0)

## Loading the dataset

In [ ]:
df = pd.read_excel(MACHINE_LEARNING_DATASET_PATH)
train_df, valid_df, test_df = split_data(df)
train, x_train, train_labels = scale_dataset(train_df, oversample=True)
valid, x_val, val_labels = scale_dataset(valid_df, oversample=False)
test, x_test, test_labels = scale_dataset(test_df, oversample=False)

print(f"Train: {x_train.shape}, Valid: {x_val.shape}, Test: {x_test.shape}")

## Utility function definitions

In [ ]:
def print_results(model: tf.keras.models.Sequential) -> Tuple:
    """
    Prints the confusion matrices for the train, validation and test data. 
    :param model: tf.keras.models.Sequential, A model that will perform the predictions. 
    :return: Tuple, containing the prediction results.
    """
    y_pred_train = model.predict(x_train).argmax(axis=1)
    print("\n###### Training ######")
    print(classification_report(train_labels, y_pred_train))
    
    y_pred_valid = model.predict(x_val).argmax(axis=1)
    print("\n###### Validation ######")
    print(classification_report(val_labels, y_pred_valid))
    
    y_pred = model.predict(x_test).argmax(axis=1)
    print("\n###### Test ######")
    print(classification_report(test_labels, y_pred))
    
    return y_pred_train, y_pred_valid, y_pred

## Load model from file

In [ ]:
model_file = "FormulaData-tf-2.15.0_Deep_Adam_8_256_0.2_0.001_2000.fnn.keras"
model = tf.keras.models.load_model(os.path.join(MODELS_PATH, model_file))

_, _, y_pred = print_results(model)

## Tuning

### Parameter tuning

In [ ]:
def tune_deep_fnn_model(df: pd.DataFrame, 
                        layers: [int], 
                        units: [int], 
                        dropout_rates: [float], 
                        learning_rates: [float], 
                        epochs: int = 200, 
                        patience: [int] = [10, 20], 
                        batch_sizes: [int] = [128]) -> Tuple[tf.keras.models.Sequential, tf.keras.callbacks.History, int]:
    """Tunes the parameters of a deep feed forward network"""
    import time
    from datetime import timedelta
    
    least_val_lost_file_name = f"tuning_least_val_loss.deep.fnn.keras"
    
    to_hh_mm_ss = lambda seconds: str(timedelta(seconds=seconds)).rsplit(".")[0]
    
    least_val_loss = float('inf')
    least_val_loss_params = []
    least_val_loss_accuracy = float('inf')
    least_val_loss_model = None  
    least_val_loss_history = None
    eta = None
    
    time_past = 0
    
    i = 1 
    max = len(layers) * len(units) * len(dropout_rates) * len(learning_rates) * len(patience) * len(batch_sizes)
    
    print("[prev: N/A] [eta: TBD]")
    
    for l in layers:
        for u in units:
            for dr in dropout_rates:
                for lr in learning_rates: 
                    for pt in patience:
                        for bt in batch_sizes:
                            start_time = time.time()
    
                            print(f"[{i}/{max}] Layers: {l}; Units: {u}; Dropout rate: {dr}; Learning rate: {lr}; Patience: {pt}; Batch size: {bt}")
                            
                            model, history, num_classes = train_deep_fnn_model(
                                df, 
                                epochs=epochs, 
                                patience=pt, 
                                layers=l, 
                                units=u,
                                dropout_rate=dr,
                                learning_rate=lr,
                                epsilon = 1e-07,
                                beta_1 = 0.9,
                                beta_2 = 0.999,
                                weight_decay = 0,
                                clipnorm = None,
                                clipvalue = None,
                                batch_size=bt,
                                verbose=0,
                                disable_save=True,
                                disable_plot_history=True,
                                disable_print_report=True)
                            
                            val_loss, val_acc = model.evaluate(x_test, test_labels)
                            print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                            if val_loss < least_val_loss:
                                model.save(os.path.join(MODELS_PATH, least_val_lost_file_name))
                                least_val_loss = val_loss
                                least_val_loss_params = [l, u, dr, lr, pt, bt]
                                least_val_loss_accuracy = val_acc
                                least_val_loss_model = model
                                least_val_loss_history = history
                                
                            duration = time.time() - start_time
                            time_past += duration
                            avg_duration = time_past / i
                            eta = time_past + avg_duration * (max - i)
                            
                            print(f"\n[eta: {to_hh_mm_ss(time_past)}/{to_hh_mm_ss(eta)}] [prev: {to_hh_mm_ss(duration)}] [avg: {to_hh_mm_ss(avg_duration)}]")
                                
                            i += 1
    
    l, u, dr, lr, pt, bt = least_val_loss_params
    print("\nLeast validation loss:")              
    print(f"\tParams:\t {{Layers: {l}; Units: {u}; Dropout rate: {dr}; Learning rate: {lr}; Patience: {pt}; Batch size: {bt}}}")
    print("\tLoss:\t", least_val_loss)
    print("\tAccuracy:\t", least_val_loss_accuracy)
    
    best_model_file_name = f"{VERSION}_Adam_{l}_{u}_{dr}_{lr}_{epochs}_{pt}_{bt}_{least_val_loss}.deep_fnn.keras"
    os.rename(
        os.path.join(MODELS_PATH, least_val_lost_file_name), 
        os.path.join(MODELS_PATH, best_model_file_name))
    print(f"\nModel has been saved as '{best_model_file_name}'")
    
    plot_history(least_val_loss_history, num_classes)
    
    print_results(model)
    
    return least_val_loss_model, least_val_loss_history, num_classes


In [ ]:
model, history, num_classes = tune_deep_fnn_model(
        df=df,
        epochs=1000,
        # Layers: 1, 2, 3, 4, 5, 6, 7
        layers=[2, 4],
        # Units: 8, 16, 32, 64, 96, 128, 160, 192
        units=[128, 256, 512],
        # Dropout rates: 0.2, 0.3, 0.4, 0.5
        dropout_rates=[0.2], 
        # Learning rates: # 0.0001, 0.0005, 0.00075, 0.001, 0.00125, 0.0015, 0.00175, 0.002
        learning_rates=[0.001, 0.0015, 0.00175],   
        # Patience: 10, 20
        patience=[10, 20, 30],
        # Batch sizes: 10, 50, 100, 128, 150, 200, 250
        batch_sizes=[128]  
    )

# Least validation loss:
# 	Params:	 {Layers: 4; Units: 512; Dropout rate: 0.2; Learning rate: 0.0015; Patience: 10; Batch size: 128}
# 	Loss:	 0.5589165091514587
# 	Accuracy:	 0.7745163440704346

## Manual Tuning

In [ ]:
model, _, num_classes = train_deep_fnn_model(
                        df, 
                        epochs=2000, 
                        patience=5, 
                        layers=8, 
                        units=256,
                        dropout_rate=0.2,
                        learning_rate=0.001,
                        verbose=2,
                        batch_size=32,
                        disable_print_report=True)
_, _, y_pred = print_results(model)

In [ ]:
distribution = get_distribution(test_df, y_pred)
# print(distribution)
plot_distribution(distribution)

### 2.4 Shap

In [ ]:
explainer = shap.KernelExplainer(model.predict, x_train)
shap_values = explainer.shap_values(shap.sample(x_test, 20), nsamples=100, random_state=41) # default of nsamples = 2 * X.shape[1] + 2048 = 2066 
# explainer.save()

from configs.enums import RISKCLASSIFICATIONS
feature_names = df.columns.tolist()
feature_names.remove(Column.COUNTRY_RISK)

In [ ]:
shap.summary_plot(shap_values, x_test, 
                  feature_names=feature_names,
                  class_names=RISKCLASSIFICATIONS.get_names())